In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Read Data Function
def read_data(filename):
    
    # Retrieve Data
    data = pd.read_csv(filename)
    
    # Drop Columns
    clean = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Date'])
    
    # Drop Last Row
    clean = clean[:-1].copy()
    
    # Drop missing data
    clean = clean.dropna()
    
    # Rename Columns
    cols = ['Year', 'Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']
    clean.columns = cols
    
    return clean

In [3]:
# Run Model (Machine Learning)
def run_model(clean):
    
    # Create Lag Column
    clean['nextSF6'] = clean.shift(1)['SF6']
    
    # Set Training and Testing Split
    training = clean.loc[(clean['Year']<2010) & (clean['Year']>1990)]
    testing = clean.loc[clean['Year']>=2010]

    # Set Variables
    X_train = training[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_train = training['nextSF6']
    X_test = testing[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_test = testing['nextSF6']
    X = clean[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y = clean['nextSF6']

    # Linear Regression Model
    from sklearn.linear_model import LinearRegression
    
    model = LinearRegression()

    # Train Modele
    model.fit(X_train, y_train)

    # Score for 1991-2009
    print(f"Train Score: {model.score(X_train, y_train)}")

    # Score for 2010+
    print(f"Test Score: {model.score(X_test, y_test)}")

    # Predictions vs Actual
    predictions = model.predict(X)
    print(f"Actual: {y[1]}")
    print(f"Predicted: {predictions[1]}")
    print(f"Error: {predictions[1]-y[1]}")

    return pd.DataFrame({"Year": clean["Year"],
                         "Predicted": predictions,
                         "Actual": y,
                         "Error": predictions - y,
                         "Country": country})

In [4]:
final = pd.DataFrame(columns = ["Year", "Predicted", "Actual", "Error", "Country"])

for filename in os.listdir("Cleaning/Combined"):
    if ".csv" in filename:
        
        print()
        
        country = filename.replace('.csv', '')
        
        print(country)
        
        clean = read_data("Cleaning/Combined/" + filename)
        
        try:
            output = run_model(clean)
            final = final.append(output)
        
        except:
            print("Error")


Australia
Train Score: 0.9004757959418043
Test Score: -7.018283343716471
Actual: 220.55969434898398
Predicted: 220.3432147959221
Error: -0.21647955306187328

Canada
Train Score: 0.8553434349006586
Test Score: -4286.891421006179
Actual: 3225.9221983891
Predicted: 3633.2850270720082
Error: 407.36282868290846

Japan
Train Score: 0.6089509412233147
Test Score: -24.178834470040442
Actual: 408.351179829816
Predicted: 367.91463506246873
Error: -40.43654476734724

New Zealand
Train Score: 0.8560166366427845
Test Score: -84.04350140624479
Actual: 206.7048
Predicted: 149.70832284645803
Error: -56.996477153541974

Russia
Train Score: 0.9784316759445517
Test Score: -8.737485475935447
Actual: 0.47496048
Predicted: 1.1416700288932056
Error: 0.6667095488932056

Spain
Train Score: 0.8941209357889103
Test Score: -4.973006038241824
Actual: 137.0062482642
Predicted: 138.34980816895268
Error: 1.3435599047526807

Switzerland
Train Score: 0.2798560560257032
Test Score: -31.89164648505775
Actual: 101.728948

In [5]:
# Export 
final.to_csv('Resources/ML_SF6.csv')